<a href="https://colab.research.google.com/github/micheleguidaa/robust-organ-scheduler/blob/main/robust_organ_scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pianificazione Robusta degli orari di prelievo per la minimizzazione dei ritardi di trasporto negli interventi di trapianto d’organo
==============


In [ ]:
import numpy as np
from datetime import datetime, date, timedelta

def parse_orario(s: str) -> datetime:
    t = datetime.strptime(s, "%H:%M").time()
    return datetime.combine(date.today(), t)

orari_possibili_inizio_intervento_raw = ["10:30", "12:00"]
orari_possibili_inizio_intervento = [parse_orario(o) for o in orari_possibili_inizio_intervento_raw]

# Durate dell'intervento espresse in minuti
durata_min = 0
durata_max = 10
durata_med = (durata_min + durata_max) // 2

durate_possibili_intervento = [
    timedelta(minutes=durata_min),
    timedelta(minutes=durata_med),
    timedelta(minutes=durata_max),
]

organi = ["cuore", "polmoni", "rene_sx", "rene_dx", "fegato"]

orari_partenza = {
    "cuore":   [parse_orario("10:45"), parse_orario("12:10")],
    "polmoni": [parse_orario("11:00"), parse_orario("12:10")],
    "rene_sx": [parse_orario("10:40"), parse_orario("12:10")],
    "rene_dx": [parse_orario("10:50"), parse_orario("12:15")],
    "fegato":  [parse_orario("11:10"), parse_orario("13:00")],
}

# Pesi che determinano l'importanza/urgenza associata a ciascun organo
pesi = {"cuore": 10, "polmoni": 9, "rene_sx": 2, "rene_dx": 2, "fegato": 2}

# Problema di minimax: minimizziamo il massimo delle attese pesate
funzione_obiettivo = np.inf
migliore_orario_inizio_intervento = None

for orario_inizio_intervento in orari_possibili_inizio_intervento:
    # per questo orario, prendiamo il peggiore caso tra le durate
    somma_pesata_attese_max = -np.inf

    for durata_intervento in durate_possibili_intervento:
        somma_pesata_attese = 0.0

        for organo in organi:
            orario_fine_intervento = orario_inizio_intervento + durata_intervento

            partenze_valide = [
                op for op in orari_partenza[organo]
                if op >= orario_fine_intervento
            ]

            if not partenze_valide:
                somma_pesata_attese = np.inf
                break

            prossima_partenza = min(partenze_valide)
            attesa = prossima_partenza - orario_fine_intervento
            attesa_minuti = attesa.total_seconds() / 60

            somma_pesata_attese += pesi[organo] * attesa_minuti

        # aggiorniamo il massimo (scenario peggiore) per questo orario
        if somma_pesata_attese > somma_pesata_attese_max:
            somma_pesata_attese_max = somma_pesata_attese

    # ora confrontiamo il massimo di questo orario con il minimo globale
    if somma_pesata_attese_max < funzione_obiettivo:
        funzione_obiettivo = somma_pesata_attese_max
        migliore_orario_inizio_intervento = orario_inizio_intervento

print("Valore funzione obiettivo (minimax):", funzione_obiettivo)

if migliore_orario_inizio_intervento is not None:
    print("Miglior orario inizio intervento:",
          migliore_orario_inizio_intervento.time())
else:
    print("Miglior orario inizio intervento: nessuna soluzione valida")


Valore funzione obiettivo (minimax): 360.0
Miglior orario inizio intervento: 12:00:00
